URL : https://github.com/nxs5899/end-to-end-Machine-Learning-model-with-MLlib-in-pySpark/blob/master/MLlib_pySpark.ipynb

URL : https://towardsdatascience.com/build-an-end-to-end-machine-learning-model-with-mllib-in-pyspark-4917bdf289c5

In [1]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc.version

'3.1.1'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
       .master("local") \
       .appName("Athulspark") \
       .config("spark.some.config.option", "some-value") \
       .getOrCreate()

In [3]:
new_df = spark.read.csv('athul_test.csv', header=True, inferSchema=True)
new_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- rotationRate: double (nullable = true)
 |-- userAcceleration: double (nullable = true)
 |-- act: double (nullable = true)
 |-- id: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)
 |-- age: double (nullable = true)
 |-- gender: double (nullable = true)
 |-- trial: double (nullable = true)



In [4]:
act_lbl = ["Sat", "Stand-Up", "Downstairs", "Upstairs", "Walking", "Jogging"]

In [5]:
drop_col = ['_c0', 'id', 'trial']

new_df = new_df.select([column for column in new_df.columns if column not in drop_col])
new_df = new_df.withColumnRenamed('act', 'label')
new_df.groupby('label').count().toPandas()

,label,count
0,0.0,234657
1,1.0,224816
2,4.0,158645
3,3.0,58204
4,2.0,50246
5,5.0,104327


In [15]:
# ! pip install seaborn
# ! pip install matplotlib

In [20]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline

df_pd = new_df.toPandas()
print(len(df_pd))

# plt.figure(figsize=(12,10))
# sns.countplot(x='label', data=df_pd, order=df_pd['label'].value_counts().index)

830895


In [21]:
import pandas as pd

pd.DataFrame(new_df.take(10), columns= new_df.columns)

,rotationRate,userAcceleration,label,weight,height,age,gender
0,0.010253,0.006959,0.0,102.0,188.0,46.0,1.0
1,0.010920,0.010673,0.0,102.0,188.0,46.0,1.0
2,0.008377,0.007010,0.0,102.0,188.0,46.0,1.0
3,0.006555,0.014892,0.0,102.0,188.0,46.0,1.0
4,0.007724,0.013001,0.0,102.0,188.0,46.0,1.0
5,0.008544,0.008358,0.0,102.0,188.0,46.0,1.0
6,0.010706,0.007313,0.0,102.0,188.0,46.0,1.0
7,0.006017,0.011407,0.0,102.0,188.0,46.0,1.0
8,0.010136,0.008716,0.0,102.0,188.0,46.0,1.0
9,0.011316,0.010180,0.0,102.0,188.0,46.0,1.0


In [22]:
cat_cols = [item[0] for item in new_df.dtypes if item[1].startswith('string')] 
print(str(len(cat_cols)) + '  categorical features')

num_cols = [item[0] for item in new_df.dtypes if item[1].startswith('int') | item[1].startswith('double')][1:]
print(str(len(num_cols)) + '  numerical features')

0  categorical features
6  numerical features


In [23]:
def info_missing_table(df_pd):
    """Input pandas dataframe and Return columns with missing value and percentage"""
    mis_val = df_pd.isnull().sum() #count total of null in each columns in dataframe
#count percentage of null in each columns
    mis_val_percent = 100 * df_pd.isnull().sum() / len(df_pd) 
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1) 
 #join to left (as column) between mis_val and mis_val_percent
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'}) 
#rename columns in table
    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1) 
        
    print ("Your selected dataframe has " + str(df_pd.shape[1]) + " columns.\n"    #.shape[1] : just view total columns in dataframe  
    "There are " + str(mis_val_table_ren_columns.shape[0]) +              
    " columns that have missing values.") #.shape[0] : just view total rows in dataframe
    return mis_val_table_ren_columns

In [24]:
missings = info_missing_table(df_pd)
missings

Your selected dataframe has 7 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [25]:
def count_missings(spark_df):
    null_counts = []        
    for col in spark_df.dtypes:    
        cname = col[0]     
        ctype = col[1]      
        nulls = spark_df.where( spark_df[cname].isNull()).count() #check count of null in column name
        result = tuple([cname, nulls])  #new tuple, (column name, null count)
        null_counts.append(result)      #put the new tuple in our result list
    null_counts=[(x,y) for (x,y) in null_counts if y!=0]  #view just columns that have missing values
    return null_counts

In [26]:
miss_counts = count_missings(new_df)
miss_counts

[]

In [ ]:
# # adding the new column weights and fill it with ratios
# from pyspark.sql.functions import when

# ratio = 0.91
# def weight_balance(labels):
#     return when(labels == 1, ratio).otherwise(1*(1-ratio))

# new_df = new_df.withColumn('weights', weight_balance(col('label')))

In [27]:
pd.DataFrame(new_df.take(10), columns= new_df.columns)

,rotationRate,userAcceleration,label,weight,height,age,gender
0,0.010253,0.006959,0.0,102.0,188.0,46.0,1.0
1,0.010920,0.010673,0.0,102.0,188.0,46.0,1.0
2,0.008377,0.007010,0.0,102.0,188.0,46.0,1.0
3,0.006555,0.014892,0.0,102.0,188.0,46.0,1.0
4,0.007724,0.013001,0.0,102.0,188.0,46.0,1.0
5,0.008544,0.008358,0.0,102.0,188.0,46.0,1.0
6,0.010706,0.007313,0.0,102.0,188.0,46.0,1.0
7,0.006017,0.011407,0.0,102.0,188.0,46.0,1.0
8,0.010136,0.008716,0.0,102.0,188.0,46.0,1.0
9,0.011316,0.010180,0.0,102.0,188.0,46.0,1.0


In [28]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

stages = []
for categoricalCol in cat_cols:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

assemblerInputs = [c + "classVec" for c in cat_cols] + num_cols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [29]:
from pyspark.ml import Pipeline

cols = new_df.columns
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(new_df)
new_df = pipelineModel.transform(new_df)

In [30]:
selectedCols = ['features']+cols
new_df = new_df.select(selectedCols)
pd.DataFrame(new_df.take(5), columns=new_df.columns)

,features,rotationRate,userAcceleration,label,weight,height,age,gender
0,"[0.006959199379238966, 0.0, 102.0, 188.0, 46.0...",0.010253,0.006959,0.0,102.0,188.0,46.0,1.0
1,"[0.010672920359489243, 0.0, 102.0, 188.0, 46.0...",0.010920,0.010673,0.0,102.0,188.0,46.0,1.0
2,"[0.0070096587648757905, 0.0, 102.0, 188.0, 46....",0.008377,0.007010,0.0,102.0,188.0,46.0,1.0
3,"[0.014892331247994722, 0.0, 102.0, 188.0, 46.0...",0.006555,0.014892,0.0,102.0,188.0,46.0,1.0
4,"[0.013001225519157802, 0.0, 102.0, 188.0, 46.0...",0.007724,0.013001,0.0,102.0,188.0,46.0,1.0


In [31]:
# split the data into trainign and testin sets

train, test = new_df.randomSplit([0.80, 0.20], seed = 42)
print(train.count())
print(test.count())

664598
166297


## LogisticRegression

In [32]:
# first we check how LogisticRegression perform 
from pyspark.ml.classification import LogisticRegression

LR = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=15)
LR_model = LR.fit(train)

## Decision Tree

In [33]:
from pyspark.ml.classification import DecisionTreeClassifier

# Train a DecisionTree model.
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label')
DT_model = dt.fit(train)

## Random Forest

In [34]:
from pyspark.ml.classification import RandomForestClassifier

# Train a RandomForestClassifier model.
rf = RandomForestClassifier(numTrees=3, maxDepth=2, labelCol="label", seed=42)
RF_model = rf.fit(train)

## Naive Bayes

In [35]:
from pyspark.ml.classification import NaiveBayes

# Train a NaiveBayes model.
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
NB_model = nb.fit(train)

# Gradio UI

In [ ]:
# ! pip install gradio

In [36]:
inputCols = ["rotationRate", "userAcceleration","weight", "height", "age", "gender"]

outputCol = "features"

In [37]:
import numpy as np
import pandas as pd

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [38]:
def model_selector(algo):
    model = LR_model
    if algo == "Logistic regression":
        model = LR_model
    elif algo == "Decision Tree":
        model = DT_model
    elif algo == "Naive Bayes":
        model = NB_model
    elif algo == "Random Forest":
        model = RF_model
    return model

In [39]:
def get_input(device):
    acc1 = 0
    gyro1 = 0
    acc2 = 0
    gyro2 = 0
    
    if device == "Device 1":
        acc1 = 0.010253
        gyro1 = 0.006959
    elif device == "Device 2":
        acc2 = 1.6289638410434408
        gyro2 = 3.8651705114155055
    else:
        acc1 = 0.010253
        gyro1 = 0.006959
        acc2 = 1.6289638410434408
        gyro2 = 3.8651705114155055
        
    return gyro1, acc1, gyro2, acc2

In [40]:
def get_prediction(device, algo, gender, age, height, weight):
    
    if gender ==  "Male":
        gender = 0
    else:
        gender = 1
        
    print(device, algo, gender, age, height, weight)
    
    prediction1 = None
    prediction2 = None
    
    if device == "Device 1":
        print("Device 1 Selected")
        gyro, acc, _ , _ = get_input(device)
        df = spark.createDataFrame([(gyro,acc,weight, height, age, gender)], inputCols)
        df_va = VectorAssembler(inputCols = inputCols, outputCol = outputCol)
        df = df_va.transform(df)
        model = model_selector(algo)
        prediction = model.transform(df)
        result_pdf = prediction.select("*").toPandas()
        prediction1 = result_pdf.prediction[0]
        print(gyro, acc)
        
    elif device == "Device 2":
        print("Device 2 Selected")
        _ , _ , gyro, acc = get_input(device)
        df = spark.createDataFrame([(gyro,acc,weight, height, age, gender)], inputCols)
        df_va = VectorAssembler(inputCols = inputCols, outputCol = outputCol)
        df = df_va.transform(df)
        model = model_selector(algo)
        prediction = model.transform(df)
        result_pdf = prediction.select("*").toPandas()
        prediction2 = result_pdf.prediction[0]
        print(gyro, acc)
        
    else:
        print("Device 1 & 2")
        gyro1, acc1, gyro2, acc2 = get_input(device)
        
        df = spark.createDataFrame([(gyro1, acc1, weight, height, age, gender)], inputCols)
        df_va = VectorAssembler(inputCols = inputCols, outputCol = outputCol)
        df = df_va.transform(df)
        model = model_selector(algo)
        prediction = model.transform(df)
        result_pdf = prediction.select("*").toPandas()
        prediction1 = result_pdf.prediction[0]
        print(gyro1, acc1)
        
        df = spark.createDataFrame([(gyro2,acc2,weight, height, age, gender)], inputCols)
        df_va = VectorAssembler(inputCols = inputCols, outputCol = outputCol)
        df = df_va.transform(df)
        model = model_selector(algo)
        prediction = model.transform(df)
        result_pdf = prediction.select("*").toPandas()
        prediction2 = result_pdf.prediction[0]
        print(gyro2, acc2)
        

    return prediction1, prediction2

In [41]:
import gradio as gr

img = gr.inputs.Image()

title = "Human Activity Recognition"

device = gr.inputs.Dropdown(choices = ["Device 1", "Device 2", "Device 1 & 2"], type="value", label="Select Device")

algo = gr.inputs.Radio(["Logistic Regression", "Decision Tree", "Naive Bayes", "Random Forest"], label="Choose Algorithm")

gender = gr.inputs.CheckboxGroup(choices = ["Male", "Female"], type="value", label="Gender")
age = gr.inputs.Slider(minimum = 20, maximum=80, step=1, default=25, label="Age")
height = gr.inputs.Slider(minimum = 100, maximum=200, step=1, default=180, label="Height")
weight = gr.inputs.Slider(minimum = 40, maximum=90, step=1, default=74, label="Weight")
        
prediction1 = gr.outputs.Textbox(type="auto", label="Device 1")
prediction2 = gr.outputs.Textbox(type="auto", label="Device 2")

gr.Interface(get_prediction, inputs = [device, algo, gender, age, height, weight],   
             outputs = [prediction1, prediction2], 
             title=title,
             live=False).launch(share=True)

Running locally at: http://127.0.0.1:7860/
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://56994.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://56994.gradio.app')

Device 1 Logistic Regression 1 25 180 74
Device 1 Selected


c:\sparkenv\opt\spark-3.1.1-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


0.006959 0.010253
Device 2 Logistic Regression 1 25 180 74
Device 2 Selected
3.8651705114155055 1.6289638410434408
Device 1 & 2 Logistic Regression 1 25 180 74
Device 1 & 2
0.006959 0.010253
3.8651705114155055 1.6289638410434408
